In [1]:
import sys
sys.path.append("./python")
import mpt
import numpy as np
import time
from tic_toc_timer import tic, toc


In [2]:
# Load Data
data_dir="./data/"
batch_size=100
train_dataset = mpt.data.MNISTDataset(
    data_dir+"/train-images-idx3-ubyte.gz",
    data_dir+"/train-labels-idx1-ubyte.gz"
)
test_dataset = mpt.data.MNISTDataset(
    data_dir+"/t10k-images-idx3-ubyte.gz",
    data_dir+"/t10k-labels-idx1-ubyte.gz"
)
train_dataloader = mpt.data.DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True
)
test_dataloader = mpt.data.DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=True
)


In [3]:
# Define network
import mpt.nn as nn

def simple_nn():
    return nn.Sequential(
        nn.Linear(784, 20),
        nn.Linear(20, 10)
    )
def loss_err(h, y):
    """ Helper function to compute both loss and error"""
    lossModule = nn.SoftmaxLoss()
    return (
        lossModule.forward(h, y),
        np.sum(h.numpy().argmax(axis=1) != y.numpy(), dtype=np.float32)
    )

In [10]:
#epoch
def epoch(dataloader: mpt.data.DataLoader,
          model: mpt.nn.Module,
          opt: mpt.optim.Optimizer = None):
    np.random.seed(4)

    if opt is None:
        model.eval()
    else :
        model.train()
    loss = 0
    err = 0
    i = 0
    num_sample = 0
    for _, data in enumerate(dataloader):
        imgs = data[0]
        labels = data[1]
        i += 1
        forwardRes = model.forward(imgs)
        iLoss, iError = loss_err(forwardRes, labels)
        loss += iLoss.numpy()[0]
        err += iError
        num_sample += labels.shape[0]
        if opt is not None:
            iLoss.backward()
            opt.step()
        print("{:.2f}% |   {:.5f}".format((err/num_sample)*100,loss/i))

    return (err/num_sample, loss/i)
    

In [11]:
# Train
train_acc, train_loss = None, None
model = simple_nn()
opt = mpt.optim.SGD(model.parameters(), lr=0.1, weight_decay=0.001)
print("Err | Loss")
for _ in range(10):
    train_acc, train_loss = epoch(train_dataloader, model=model, opt=opt)
test_acc, test_loss = epoch(test_dataloader, model=model)
print()
print("{:.5f} |   {:.5f}".format(test_acc, test_loss))

Err | Loss
93.00% |   2.50175
83.50% |   2.33086
79.33% |   2.24553
74.75% |   2.15502
71.20% |   2.07882
67.67% |   2.00634
63.86% |   1.93951
61.00% |   1.88291
57.78% |   1.79896
55.60% |   1.75631
53.73% |   1.71284
52.17% |   1.67485
50.23% |   1.63008
49.07% |   1.60569
47.53% |   1.56638
46.00% |   1.53724
45.06% |   1.50852
44.11% |   1.48187
43.37% |   1.45998
42.20% |   1.43327
41.33% |   1.41159
40.91% |   1.39114
40.39% |   1.37336
39.50% |   1.34843
38.88% |   1.32871
37.92% |   1.30506
37.26% |   1.28609
36.71% |   1.27104
36.28% |   1.25793
35.70% |   1.24364
35.16% |   1.22610
34.78% |   1.21532
34.33% |   1.20553
33.91% |   1.19249
33.46% |   1.17953
33.00% |   1.16762
32.76% |   1.15648
32.21% |   1.14335
31.79% |   1.13026
31.52% |   1.11965
31.15% |   1.10754
30.83% |   1.09665
30.35% |   1.08365
30.11% |   1.07475
29.76% |   1.06466
29.37% |   1.05444
29.26% |   1.04744
28.94% |   1.03789
28.69% |   1.02936
28.50% |   1.02087
28.37% |   1.01370
28.12% |   1.00638
2

KeyboardInterrupt: 

In [61]:
testImg=mpt.Tensor(test_dataset[0][0]).reshape((-1,1)).transpose()
testLabel=mpt.Tensor(test_dataset[0][1])
def loss_err(h, y):
    lossModule = nn.SoftmaxLoss()
    return (
        lossModule.forward(h, y),
        np.sum(h.numpy().argmax(axis=1) != y.numpy(), dtype=np.float32)
    )

In [75]:
h=model.forward(testImg)
y=testLabel
print(np.sum(h.numpy().argmax(axis=1) != y.numpy(), dtype=np.float32))

0.0


In [80]:
h2=net.forward(testImg)
y2=testLabel
print(h2)
print(h2.numpy().argmax(axis=1) != y2.numpy())
print(np.sum(h2.numpy().argmax(axis=1) != y2.numpy(), dtype=np.float32))

[[ 0.38178068 -0.45340216  0.46996555  0.6675435  -0.31004506  0.46440935
  -0.59473586  0.2853446  -1.3865604  -0.27761897]]
[ True]
1.0


In [7]:

a=mpt.Tensor(np.random.randn(320,320),device=mpt.cpu())
b=mpt.Tensor(np.random.randn(320,320),device=mpt.cpu())
tic()
res=mpt.ops.matmul(a,b)
print(f"The total time elapsed is {toc()}")
print(res,res.device,res.dtype)

c=np.random.randn(320,320).astype('f')
d=np.random.randn(320,320).astype('f')
res2=np.ndarray((320,320))
tic()
for i in range(320):
    for j in range(320):
        thissum=0
        for k in range(320):
            thissum+=c[i,j]*d[j,k]
        res2[i,k]=thissum
# print(c)
# print(d)
print(f"The total time elapsed is {toc()}")

The total time elapsed is 0:00:00.908593
[[  2.8811848 -12.453715   -4.7072725 ... -13.744873  -20.835342
   -8.44306  ]
 [-47.7621     -3.5861657   2.5874667 ...  -1.7962167  13.986323
  -16.019613 ]
 [ 37.928497   20.674397  -14.926062  ...   9.745806   -5.121373
   12.80354  ]
 ...
 [ -5.649159   10.830176    9.132078  ...  -3.7227452 -12.196058
  -17.02923  ]
 [ 15.431809   11.98895    10.8527155 ... -13.062768   -7.806604
   14.733793 ]
 [-17.335762   30.071396  -11.187022  ...   8.875761   27.101534
   14.943199 ]] cpu() float32
The total time elapsed is 0:00:19.478694
